In [1]:
import random
random.seed(1)
import datetime
import heapq
from collections import namedtuple

<!-- **CLASSES** -->

In [2]:
class Item:
    def __init__(self, item: str, utility: int):
        self.item = item
        self.utility = utility
        self._twu = 0

    @property
    def twu(self) -> int:
        return self._twu

    @twu.setter
    def twu(self, value: int) -> None:
        self._twu = value

    def __repr__(self):
        return f"{self.item}"
        # return f"({self.item}, {self.utility}, {self.twu})"

    def __eq__(self, other):
        if isinstance(other, Item):
            return self.item == other.item and self.utility == other.utility
        return False

    def __hash__(self):
        return hash((self.item, self.utility))


def check_order_condition(a: Item, b: Item) -> bool:
    """_summary_

    Args:
        a (Item): an item
        b (Item): an item

    Returns:
        bool: return b < a
    """
    if a.utility * b.utility < 0:
        return a.utility < b.utility
    elif a.utility * b.utility > 0:
        if a.twu == b.twu:
            if a.utility == b.utility:
                return a.item > b.item
            return b.utility > a.utility
        return a.twu > b.twu
    return False


def check_order_item_and_set(item: Item, item_set: set[Item]) -> bool:
    """_summary_
    This function is used to check an item > item-set or not.
    Example: a > {b, c}
    Args:
        item (Item): an item
        item_set (set[Item]): an item set

    Returns:
        bool: return item > item_set
    """
    for i in item_set:
        if check_order_condition(item, i) == False:
            return False
    return True


class TransItem:
    def __init__(self, item: Item, quantity: int, probability: float):
        self.item = item
        self.quantity = quantity
        self.probability = probability

    def __repr__(self):
        return f"{self.item},{self.quantity},{self.probability}"

    def get_total_probability(self):
        return self.quantity * self.probability


class Transaction:
    def __init__(self, id: int, trans_items: list[TransItem]):
        self.id = id
        self.trans_items_list = [trans_item.item for trans_item in trans_items]
        self.trans_items_dict = {
            trans_item.item: (trans_item.quantity, trans_item.probability)
            for trans_item in trans_items
        }

    def __repr__(self):
        return f"Transaction(id={self.id}, items={list(self.trans_items_dict.keys())}, quantities={list(self.trans_items_dict.values())})"

    def contains_item_set(self, item_set: set[Item]) -> bool:
        # Check directly against the dictionary keys
        return item_set.issubset(self.trans_items_dict.keys())

    def get_quantity_of_item(self, item: Item) -> int:
        # Access quantity directly from the dictionary
        return self.trans_items_dict.get(item, (0, 0))[0]

    def get_probability_of_item(self, item: Item) -> float:
        # Access probability directly from the dictionary
        return self.trans_items_dict.get(item, (0, 0))[1]

    def get_positive_utility_of_trans(self):
        put = 0
        positive_item = {item for item in self.get_items() if item.utility > 0}
        for item in positive_item:
            put += item.utility * self.get_quantity_of_item(item)
        return put

    def get_items(self) -> set[Item]:
        return set(self.trans_items_dict.keys())

    def get_probability_of_item_set(self, item_set: set[Item]) -> float:
        if not self.contains_item_set(item_set):
            return 0.0
        total_probability = 1.0
        for item in item_set:
            total_probability *= self.get_probability_of_item(item)
        return total_probability

    def _calculate_utility(self, item_set: set[Item], condition: callable) -> int:
        total_utility = 0
        for item in item_set:
            quantity = self.get_quantity_of_item(item)
            if condition(item.utility):
                total_utility += item.utility * quantity
        return total_utility

    def get_positive_utility_of_item_set(self, item_set: set[Item]) -> int:
        return self._calculate_utility(item_set, lambda utility: utility > 0)

    def get_negative_utility_of_item_set(self, item_set: set[Item]) -> int:
        return self._calculate_utility(item_set, lambda utility: utility < 0)

    def get_utility_of_item_set(self, item_set: set[Item]) -> int:
        return self._calculate_utility(item_set, lambda utility: True)

    def sort_trans_items_by_twu_and_utility(self) -> None:
        def sort_key(item: Item) -> tuple:
            return (0 if item.utility > 0 else 1, item.twu, -item.utility, item.item)
        self.trans_items_list.sort(key=sort_key)

    def get_remaining_utility_of_item_set(self, item_set: list[Item]) -> int:
        ru = 0
        if item_set:
            last_item = item_set[-1]
            index = self.trans_items_list.index(last_item)
            for i in range(index + 1, len(self.trans_items_list)):
                item = self.trans_items_list[i]
                if item.utility > 0 and item not in item_set:
                    quantity = self.trans_items_dict[item][0]
                    ru += item.utility * quantity
        return ru

class PriorityQueue:
    def __init__(self, max_size: int):
        self.max_size = max_size
        self.heap: list[tuple[int, set[Item]]] = []
        self.item_sets: set[frozenset] = set()

    def push(self, utility: int, item_set: set):
        fs_item_set = frozenset(item_set)
        if fs_item_set in self.item_sets:
            return

        if len(self.heap) < self.max_size:
            heapq.heappush(self.heap, (utility, item_set))
            self.item_sets.add(fs_item_set)
        else:
            if utility > self.heap[0][0]:
                removed = heapq.heappushpop(self.heap, (utility, item_set))
                self.item_sets.remove(frozenset(removed[1]))
                self.item_sets.add(fs_item_set)

    def get_min_utility(self) -> int:
        if self.heap:
            return self.heap[0][0]
        return float("-inf")

    def sort(self):
        return sorted(self.heap, reverse=True)

    def print_items(self):
        for utility, item_set in self.sort():
            print(f"{item_set}: {utility}")


from dataclasses import dataclass


@dataclass
class Utilities:
    tid: int
    pro: float
    pu: int
    nu: int
    ru: int


class PNUList:
    def __init__(self, items: set[Item], utility_values: list[Utilities]):
        self.items = items
        self.utility_values = utility_values

    def get_ru(self) -> int:
        ru = 0
        for i in self.utility_values:
            ru += i.ru
        return ru

    def get_pu(self) -> int:
        pu = 0
        for i in self.utility_values:
            pu += i.pu
        return pu

    def get_nu(self) -> int:
        nu = 0
        for i in self.utility_values:
            nu += i.nu
        return nu

    def get_pro(self) -> float:
        pro = 0
        for i in self.utility_values:
            pro += i.pro
        return pro

    def get_utility(self) -> int:
        u = 0
        for i in self.utility_values:
            u += i.pu + i.nu
        return u

    def __repr__(self):
        items_str = ", ".join(str(item) for item in self.items)
        utility_values_str = ", \n".join(
            str(utility) for utility in self.utility_values
        )
        return f"PNUList(\n  Items: [{items_str}]\n  Utility Values: \n[{utility_values_str}]\n)"

    def __eq__(self, other):
        if not isinstance(other, PNUList):
            return False
        return self.items == other.items and self.utility_values == other.utility_values

    def __hash__(self):
        return hash(frozenset(self.items)) ^ hash(tuple(self.utility_values))


class MList(PNUList):
    def __init__(
        self,
        items: set[Item],
        true_items: set[Item],
        prefix: PNUList,
        utility_values: list[Utilities],
        ru: int,
        pu: int,
    ):
        super().__init__(items, utility_values)
        self.true_items = true_items
        self.prefix = prefix
        self.ru = ru
        self.pu = pu

    def __repr__(self):
        return (
            f"MList(items={repr(self.items)}, "
            f"true_items={repr(self.true_items)}, "
            f"utility_values={repr(self.utility_values)}, "
            f"ru={self.ru}, pu={self.pu})"
        )


Periodic = namedtuple("Periodic", ["min_per", "max_per", "avg_per"])

<!-- ***Example*** -->

In [3]:
def create_transaction_dictionary(item_list: list[Item], database: list[Transaction]):
    trans_dict: dict[Item, set[Transaction]] = {item: set() for item in item_list}
    for trans in database:
        trans_items = trans.get_items()
        for item in trans_items:
            trans_dict.get(item).add(trans)
    return trans_dict

In [4]:
def sort_items_by_twu_and_utility(items: list[Item]) -> list[Item]:
    """_summary_

    Args:
        items (list[Item]): a list item

    Returns:
        list[Item]: a sorted list by two principle:
        1. positive always takes priority over negative
        2. ascending twu value
    """    
    def sort_key(item: Item) -> tuple:
        return (0 if item.utility > 0 else 1, item.twu, -item.utility, item.item)
    return sorted(items, key=sort_key)

In [5]:
def calculate_utility_of_item_set_in_database(
    item_set: set[Item], trans_dict: dict[Item, set[Transaction]]
) -> int:
    transaction_set: set[Transaction] = set()
    for item in item_set:
        if not transaction_set:
            transaction_set = set(trans_dict[item])
            continue
        transaction_set &= set(trans_dict[item])
    utility = 0
    for trans in transaction_set:
        utility += trans.get_utility_of_item_set(item_set)
    return utility

<!-- **Algorithm 1: Positive Real Item Utility strategy** -->


In [6]:
def priu_pruning(
    utility_arr: list[int],
    k: int
) -> int:
    sorted_util: list[int] = sorted(utility_arr, reverse=True)
    if not sorted_util:
        return 0
    return (
        sorted_util[- 1]
        if k > len(utility_arr)
        else sorted_util[k - 1]
    )

<!-- **Algorithm 2: Positive LIU-Exact strategy** -->


In [7]:
def pliue_strategy(lius: dict[frozenset[Item], int], k: int, current_min_util: int):
    """_summary_

    Args:
        lius (dict[frozenset[Item], int]): _description_
        k (int): _description_co
        current_min_util (int): _description_

    Returns:
        _type_: _description_
    """
    piqu_liu = list()
    for _, utility in lius.items():
        piqu_liu.append(utility)

    piqu_liu.sort(reverse=True)

    max_index = len(piqu_liu) - 1 if k > len(piqu_liu) else k - 1
    if piqu_liu[max_index] > current_min_util:
        current_min_util = piqu_liu[max_index]
    return current_min_util

In [8]:
def create_liu_dict(item_list: list, trans_dict: dict[Item, set[Transaction]]) -> dict:
    n = len(item_list)
    liu_dict: dict[frozenset[Item], int] = {}
    # Generate all pairs of items using nested loops
    for i in range(n):
        for j in range(i + 1, n):
            item1 = item_list[i]
            item2 = item_list[j]

            # Retrieve transactions containing item1 and item2
            trans1 = trans_dict.get(item1, set())
            trans2 = trans_dict.get(item2, set())

            # Find common transactions for the pair
            relevant_transactions = trans1 & trans2

            # Calculate the utility of the item pair
            utility = sum(
                trans.get_utility_of_item_set({item1, item2})
                for trans in relevant_transactions
            )

            # Add the pair and its utility to the dictionary
            liu_dict[frozenset((item1, item2))] = utility

    return liu_dict

<!-- **Algorithm 3: PLIU_LB strategy** -->


In [9]:
def pliu_lb_strategy(
    lius: dict[frozenset[Item], int],
    piqu_liu: set[int],
    ordered_list: list[Item],
    k: int,
    current_min_util: int,
    database: list[Transaction],
) -> int:
    piqu_lb_liu: set[int] = set()
    for key, value in lius.items():
        key_list = list(key)
        post_start_item: Item = (
            key_list[0]
            if check_order_condition(key_list[1], key_list[0])
            else key_list[1]
        )

        post_end_item: Item = (
            key_list[1]
            if check_order_condition(key_list[1], key_list[0])
            else key_list[0]
        )

        start_index = ordered_list.index(post_start_item)
        end_index = ordered_list.index(post_end_item)

        # sub_list = ordered_list[start_index + 1 : end_index]

        for x in range(start_index + 1, end_index):
            util_lb = value - calculate_utility_of_item_set_in_database(
                {ordered_list[x]}, database
            )
            if util_lb > current_min_util:
                piqu_lb_liu.add(util_lb)
            for y in range(x + 1, end_index):
                # print("y = " + str(ordered_list[y]))
                util_lb = (
                    value
                    - calculate_utility_of_item_set_in_database(
                        {ordered_list[x]}, database
                    )
                    - calculate_utility_of_item_set_in_database(
                        {ordered_list[y]}, database
                    )
                )
                if util_lb > current_min_util:
                    piqu_lb_liu.add(util_lb)
                for z in range(y + 1, end_index):
                    util_lb = (
                        value
                        - calculate_utility_of_item_set_in_database(
                            {ordered_list[x]}, database
                        )
                        - calculate_utility_of_item_set_in_database(
                            {ordered_list[y]}, database
                        )
                        - calculate_utility_of_item_set_in_database(
                            {ordered_list[z]}, database
                        )
                    )
                    if util_lb > current_min_util:
                        piqu_lb_liu.add(util_lb)
                    for w in range(z + 1, end_index):
                        util_lb = (
                            value
                            - calculate_utility_of_item_set_in_database(
                                {ordered_list[x]}, database
                            )
                            - calculate_utility_of_item_set_in_database(
                                {ordered_list[y]}, database
                            )
                            - calculate_utility_of_item_set_in_database(
                                {ordered_list[z]}, database
                            )
                            - calculate_utility_of_item_set_in_database(
                                {ordered_list[w]}, database
                            )
                        )
                        if util_lb > current_min_util:
                            piqu_lb_liu.add(util_lb)
    piqu_all: list[int] = list(piqu_lb_liu | piqu_liu)
    if len(piqu_all) >= k:
        piqu_all = list(piqu_all)
        piqu_all.sort(reverse=True)
        current_min_util = piqu_all[0]
    return current_min_util

In [10]:
def calculate_local_utility(alpha: set[Item], item: Item, database: list[Transaction]) -> int:
    """_summary_

    Args:
        alpha (set[Item]): an item set
        item (Item): a combined item
        database (list[Transaction]): database

    Returns:
        int: return a local utility value of item set
    """    
    lu = 0
    for trans in database:
        if trans.contains_item_set(alpha | {item}):
            lu += trans.get_utility_of_item_set(
                alpha
            ) + trans.get_remaining_utility_of_item_set(alpha)
    return lu


def calculate_subtree_utility(alpha: set[Item], item: Item, db: list[Transaction]) -> int:
    """_summary_

    Args:
        alpha (set[Item]): an item set
        item (Item): a combined item
        database (list[Transaction]): database

    Returns:
        int: return subtree utility value of item set
    """    
    su = 0
    if check_order_item_and_set(item, alpha):
        for trans in db:
            if trans.contains_item_set(alpha | {item}):
                su += (
                    trans.get_utility_of_item_set(alpha)
                    + trans.get_utility_of_item_set({item})
                    + trans.get_remaining_utility_of_item_set(set(alpha | {item}))
                )
    return su

In [11]:
def calculate_utilities_of_item_set(
    item_set: set[Item], database: list[Transaction]
) -> tuple[float, int, int, int]:
    """_summary_

    Args:
        item_set (set[Item]): an item set
        database (list[Transaction]): database

    Returns:
        tuple[float, int, int, int]: return a tuple (probability, utility, remaining-utility, twu)
    """
    p: float = 0.0
    u: int = 0
    ru: int = 0
    twu: int = 0
    for trans in database:
        if trans.contains_item_set(item_set):
            p += trans.get_probability_of_item_set(item_set)
            u += trans.get_utility_of_item_set(item_set)
            ru += trans.get_remaining_utility_of_item_set(item_set)
            twu += trans.get_positive_utility_of_trans()
    return p, u, ru, twu


def calculate_probability_and_utility_of_item_set(
    item_set: set[Item], database: list[Transaction]
):
    p: float = 0.0
    u: int = 0
    for trans in database:
        if trans.contains_item_set(item_set):
            p += trans.get_probability_of_item_set(item_set)
            u += trans.get_utility_of_item_set(item_set)
    return p, u


def calculate_probability_and_utility_of_item_set_v2(
    item_set: set[Item], trans_dict: dict[Item, set[Transaction]]
):
    intersection_trans = set()
    for item in item_set:
        if not intersection_trans:
            intersection_trans = set(trans_dict.get(item, set()))
            continue
        intersection_trans &= trans_dict.get(item)
    p: float = 0.0
    u: int = 0
    for trans in intersection_trans:
        p += trans.get_probability_of_item_set(item_set)
        u += trans.get_utility_of_item_set(item_set)
    return p, u

In [12]:
def find_database_projection(item_set: set[Item], trans_dict: dict[Item, set[Transaction]]) -> list[Transaction]:
    intersection_trans = set()
    for item in item_set:
        if not intersection_trans:
            intersection_trans = set(trans_dict.get(item, set()))
            continue
        intersection_trans &= trans_dict.get(item)
    return intersection_trans

In [ ]:
def create_prob_twu_utility_bin_array(
    item_list: set[Item], database: list[Transaction]
) -> tuple[list[int], list[int], list[int], dict[Item, int]]:
    n = len(item_list)
    twu_array = [0 for _ in range(n)]
    prob_array = [0 for _ in range(n)]
    utility_array = [0 for _ in range(n)]
    item_dict = {item: idx for idx, item in enumerate(item_list)}
    for trans in database:
        ptu = trans.get_positive_utility_of_trans()
        for item in trans.get_items():
            index = item_dict.get(item, -1)
            if index != -1:
                twu_array[index] += ptu
                prob_array[index] += trans.get_probability_of_item(item)
                utility_array[index] += trans.get_quantity_of_item(item) * item.utility
    return prob_array, twu_array, utility_array, item_dict


def create_local_and_subtree_utility_bin_array(
    alpha: set[Item], item_list: set[Item], database: list[Transaction]
) -> tuple[list[int], list[int], list[float]]:
    n = len(item_list)
    lu_array = [0 for _ in range(n)]
    su_array = [0 for _ in range(n)]
    prob_array = [0.0 for _ in range(n)]
    item_dict = {item: idx for idx, item in enumerate(item_list)}
    sorted_alpha = sort_items_by_twu_and_utility(alpha)
    for trans in database:
        if trans.contains_item_set(alpha):
            alpha_util = trans.get_utility_of_item_set(alpha)
            for item in trans.get_items():
                index = item_dict.get(item, -1)
                if index != -1:
                    lu_array[index] += (alpha_util + trans.get_remaining_utility_of_item_set(sorted_alpha))
                    prob_array[index] += trans.get_probability_of_item(item)
                    if item in item_list and check_order_item_and_set(item, alpha):
                        new_sorted = list(sorted_alpha)
                        new_sorted.append(item)
                        su_array[index] += (
                            alpha_util
                            + (trans.get_quantity_of_item(item) * item.utility)
                            + trans.get_remaining_utility_of_item_set(new_sorted)
                        )
    return lu_array, su_array, prob_array


def get_value_from_bin(
    item: Item, bin: list[int | float], item_dict: dict[Item, int | float]
):
    index = item_dict.get(item, -1)
    return bin[index] if index != -1 else 0


def create_item_transId_dict(
    item_list: list[Item], db: list[Transaction]
) -> dict[Item, list[int]]:
    transaction_index = {item: list() for item in item_list}
    for trans in db:
        for item in trans.trans_items_dict.keys():
            transaction_index[item].append(trans.id)
    return transaction_index

def find_all_trans_ids_contained_item_set(
    item_set: set[Item], create_item_transId_dict: dict[Item, list[int]]
) -> list[int]:
    trans_ids = list()
    for item in item_set:
        if not trans_ids:
            trans_ids = set(create_item_transId_dict[item])
            continue
        trans_ids &= set(create_item_transId_dict[item])
    return sorted(trans_ids) if trans_ids else []


def find_min_max_avg_periodic_of_item_set(
    item_set: set[Item], db: list[Transaction], transaction_index: dict[Item, list[int]]
):
    trans_ids = find_all_trans_ids_contained_item_set(item_set, transaction_index)
    if not trans_ids:
        return 0, 0, 0

    m = len(trans_ids)
    max_per, min_per, total_per = 0, float("inf"), 0
    prev = 0
    for _, trans_id in enumerate(trans_ids):
        per = trans_id - prev
        max_per = max(max_per, per)
        min_per = min(min_per, per)
        total_per += per
        prev = trans_id
    final_per = len(db) - prev
    max_per = max(max_per, final_per)
    min_per = min(min_per, final_per)
    total_per += final_per
    avg_per = total_per / (m + 1)
    return min_per, max_per, avg_per

In [14]:
def efim_global_search(
    alpha: set[Item],
    primary: list[Item],
    secondary: list[Item],
    min_util: int,
    min_per_threshold: int,
    max_per_threshold: int,
    min_avg_threshold: int,
    max_avg_threshold: int,
    user_prob_threshold: float,
    k: int,
    item_dict: dict[Item, int],
    topk_queue: PriorityQueue,
    trans_dict: dict[Item, set[Transaction]],
    item_transId_dict: dict[Item, list[int]],
    periodic_dict: dict[Item, type[Periodic]],
    database: list[Transaction],
) -> None:

    for pri_item in primary:
        beta: set[Item] = alpha | {pri_item}
        prob, util = calculate_probability_and_utility_of_item_set_v2(beta, trans_dict)
        min_per, max_per, avg_per = find_min_max_avg_periodic_of_item_set(
            beta, database, item_transId_dict
        )
        if (
            min_per >= min_per_threshold
            and max_per <= max_per_threshold
            and avg_per >= min_avg_threshold
            and avg_per <= max_avg_threshold
            and util >= min_util
            and round(prob, 3) >= user_prob_threshold
        ):
            topk_queue.push(util, beta)
            if len(topk_queue.heap) == k:
                min_util = topk_queue.sort()[k - 1][0]

        i = secondary.index(pri_item)
        if i != -1 and i + 1 < len(secondary):
            second_item = secondary[i + 1]
            if (
                max_per > max_per_threshold
                or avg_per > max_avg_threshold
                or util < min_util
                or round(prob, 3) < user_prob_threshold
            ) and second_item.utility < 0:
                continue
        beta_dp: list[Transaction] = find_database_projection(beta, trans_dict)
        lu_arr, su_arr, prob_arr = create_local_and_subtree_utility_bin_array(
            beta, secondary, beta_dp
        )
        new_primary = list()
        new_secondary = list()
        for j in range(i + 1, len(secondary)):
            item = secondary[j]
            _, max_per, avg_per = periodic_dict.get(item, (0, 0, 0))
            if max_per <= max_per_threshold and avg_per <= max_avg_threshold and item.twu >= min_util:
                p = prob_arr[j]
                lu = lu_arr[j]
                if round(p, 3) >= user_prob_threshold and lu >= min_util:
                    new_secondary.append(item)
                    su = su_arr[j]
                    if su >= min_util:
                        new_primary.append(item)
        if new_primary and new_secondary:
            efim_global_search(
                beta,
                new_primary,
                new_secondary,
                min_util,
                min_per_threshold,
                max_per_threshold,
                min_avg_threshold,
                max_avg_threshold,
                user_prob_threshold,
                k,
                item_dict,
                topk_queue,
                trans_dict,
                item_transId_dict,
                periodic_dict,
                database
            )

In [15]:
def generate_mlist(X: PNUList, Y: PNUList, P: PNUList, du: int):
    z = X.items | Y.items
    mlist = MList(z, Y.items, P, Y.utility_values, du, 0)
    return mlist

In [16]:
def topk_mining_based_on_EFIM(
    database: list[Transaction],
    item_list: list[Item],
    k: int,
    min_per_threshold: int,
    max_per_threshold: int,
    min_avg_threshold: int,
    max_avg_threshold: int,
    min_prob: float,
) -> PriorityQueue:
    t1 = datetime.datetime.now()
    if k <= 0:
        return
    # Init alpha with empty set
    alpha: set[Item] = set()
    topk_queue = PriorityQueue(k)
    min_util = 0
    user_prob_threshold = min_prob * len(database)

    prob_arr, twu_arr, utility_arr, item_dict = create_prob_twu_utility_bin_array(
        item_list, database
    )
    # Get only positive item set
    positive_utility_list: list[Item] = [
        utility for utility in utility_arr if utility > 0
    ]

    # Update min_util = the k-th largest in utility array
    min_util: int = priu_pruning(positive_utility_list, k)

    # Create secondary list
    secondary = list()

    item_transId_dict: dict[Item, list[int]] = create_item_transId_dict(
        item_list, database
    )

    # Create periodic dictionary
    periodic_dict = {item: Periodic for item in item_list}

    for i in range(len(item_list)):
        min_per, max_per, avg_per = find_min_max_avg_periodic_of_item_set(
            {item_list[i]}, database, item_transId_dict
        )
        if (
            max_per <= max_per_threshold and avg_per <= max_avg_threshold
            and round(prob_arr[i], 3) >= user_prob_threshold
            and twu_arr[i] >= min_util
        ):
            item_list[i].twu = twu_arr[i]
            secondary.append(item_list[i])
            periodic_dict[item_list[i]] = (min_per, max_per, avg_per)

    # Create removed_list contains item unqualified
    removed_list = set(item_list).difference(secondary)

    # Remove those items in removed_list from database
    for trans in database:
        for item in removed_list:
            trans.trans_items_dict.pop(item, None)
            periodic_dict.pop(item, None)
            try:
                trans.trans_items_list.remove(item)
            except ValueError:
                pass
        trans.sort_trans_items_by_twu_and_utility()

    # Convert database (list[Transaction]) to trans_dict (dict[Item, set[Transaction]])
    trans_dict: dict[Item, set[Transaction]] = create_transaction_dictionary(
        secondary, database
    )
    # Create liu dict, to update min_util
    lius: dict[frozenset[Item], int] = create_liu_dict(secondary, trans_dict)
    min_util = pliue_strategy(lius, k, min_util)

    # Create local & subtree bin array for all item
    _, su_array, _ = create_local_and_subtree_utility_bin_array(
        alpha, secondary, database
    )
    # Create primary & secondary list
    primary: list[Item] = [
        secondary[i] for i in range(len(secondary)) if su_array[i] >= min_util
    ]
    secondary = sort_items_by_twu_and_utility(secondary)

    t2 = datetime.datetime.now()
    print("phase 1 util: " + str(min_util))
    print("phase 1 take time: " + str(t2 - t1))
    t1 = datetime.datetime.now()

    efim_global_search(
        alpha,
        primary,
        secondary,
        min_util,
        min_per_threshold,
        max_per_threshold,
        min_avg_threshold,
        max_avg_threshold,
        user_prob_threshold,
        k,
        item_dict,
        topk_queue,
        trans_dict,
        item_transId_dict,
        periodic_dict,
        database
    )
    t3 = datetime.datetime.now()
    print("phase 2 util: " + str(min_util))
    print("phase 2 take time: " + str(t3 - t2))
    return topk_queue

<!-- **BASE ON PHUI** -->


In [17]:
def find_tuple_by_trans_id(P: PNUList, target_trans_id: int) -> Utilities:
    utilities_list: list[Utilities] = P.utility_values
    for iTuple in utilities_list:
        if iTuple.tid == target_trans_id:
            return iTuple
    return None

def utility_list_construct(
    P: PNUList,
    Px: PNUList,
    Py: PNUList,
    min_util: int,
    user_prob_threshold: float,
):
    if not Px or not Py or not Px.utility_values or not Py.utility_values:
        return None

    x = Px.items
    y = Py.items
    xy = x | y
    utilities_list: list[Utilities] = []
    Pxy = PNUList(xy, utilities_list)

    y_dict = {utl.tid: utl for utl in Py.utility_values}
    p_dict = (
        {utl.tid: utl for utl in P.utility_values} if P and P.utility_values else {}
    )

    probability = Px.get_pro()
    utility = Px.get_pu() + Px.get_ru()

    for xTuple in Px.utility_values:
        yTuple = y_dict.get(xTuple.tid, None)

        if yTuple:
            if P and P.utility_values:
                pTuple = p_dict.get(xTuple.tid, None)
                if pTuple:
                    pro = 1e-10 if pTuple.pro == 0 else pTuple.pro
                    xyTuple = Utilities(
                        xTuple.tid,
                        xTuple.pro * yTuple.pro / pro,
                        xTuple.pu + yTuple.pu - pTuple.pu,
                        xTuple.nu + yTuple.nu - pTuple.nu,
                        yTuple.ru,
                    )
                    utilities_list.append(xyTuple)
            else:
                xyTuple = Utilities(
                    xTuple.tid,
                    xTuple.pro * yTuple.pro,
                    xTuple.pu + yTuple.pu,
                    xTuple.nu + yTuple.nu,
                    yTuple.ru,
                )
                utilities_list.append(xyTuple)
        else:
            probability -= xTuple.pro
            utility -= xTuple.pu + xTuple.ru

            if probability < user_prob_threshold or utility < min_util:
                return None
    return Pxy


def covl_construct(
    sorted_list: list[Item],
    eucst_dict: dict[frozenset[Item], int],
    utility_arr: list[int],
    item_index_dict: dict[Item, int],
    trans_dict: dict[Item, set[Transaction]]
):
    covl_list = list()
    for i in range(len(sorted_list)):
        x: Item = sorted_list[i]
        coverage_list = list()
        for j in range(i + 1, len(sorted_list)):
            y: Item = sorted_list[j]
            xy: set[Item] = {x, y}
            key = frozenset(xy)
            xy_twu = eucst_dict.get(key)
            if x.twu == xy_twu:
                coverage_list.append(y)
        r = len(coverage_list)
        if r == 0:
            coverage_list.append(-1)
        else:
            util: int = 0
            for z in range(0, r):
                util += calculate_utility_of_item_set_in_database(
                    {x, coverage_list[z]}, trans_dict
                )
            util -= (r - 1) * get_value_from_bin(x, utility_arr, item_index_dict)
            covl_list.append(util)
    covl_list.sort(reverse=True)
    return covl_list

In [18]:
def create_eucst_dict(
    sorted_item_list: list[Item], database: list[Transaction], trans_dict: dict[Item, set[Transaction]], min_util: int
):
    # Calculate all positive utility of transaction in database, and store in a dictionary
    transaction_twu = {
        trans: trans.get_positive_utility_of_trans() for trans in database
    }
    # Create a dictionary with key is a item and value is a list of transaction that contains item
    eucst_dict = {}
    n = len(sorted_item_list)
    for i in range(n):
        item1 = sorted_item_list[i]
        for j in range(i + 1, n):
            item2 = sorted_item_list[j]
            # Using AND operator to get list of transaction that contains item set {item1, item2}
            relevant_transactions: set[Transaction] = trans_dict.get(item1, set()) & trans_dict.get(item2, set())
            # Calculate sum utility in that transaction list, NOT in database
            twu = sum(
                transaction_twu[trans]
                for trans in relevant_transactions
            )
            if twu >= min_util:
                eucst_dict[frozenset({item1, item2})] = twu
    return eucst_dict

def update_eucst_dict(
    eucst_dict: dict[frozenset, int], min_util: int
) -> dict[frozenset, int]:
    return {key: twu for key, twu in eucst_dict.items() if twu >= min_util}

In [43]:
def phui_searching_procedure(
    PList: PNUList,
    lists: list[PNUList],
    current_min_util: int,
    min_per_threshold: int,
    max_per_threshold: int,
    min_avg_threshold: int,
    max_avg_threshold: int,
    user_prob_threshold: float,
    database: list[Transaction],
    eucs_dict: dict[frozenset[Item], int],
    item_transId_dict: dict[Item, list[int]],
    k: int,
    topk_queue: PriorityQueue,
):
    for i in range(len(lists)):
        XList: PNUList = lists[i]
        XList_utility = XList.get_utility()
        XList_prob = XList.get_pro()
        XList_ru = XList.get_ru()
        min_per, max_per, avg_per = find_min_max_avg_periodic_of_item_set(
            XList.items, database, item_transId_dict
        )
        if (
            min_per >= min_per_threshold
            and max_per <= max_per_threshold
            and avg_per >= min_avg_threshold
            and avg_per <= max_avg_threshold
            and XList_utility >= current_min_util
            and round(XList_prob, 3) >= user_prob_threshold
        ):
            topk_queue.push(XList_utility, XList.items)
            if len(topk_queue.heap) == k:
                current_min_util = topk_queue.sort()[k - 1][0]

        if (
            max_per <= max_per_threshold
            and avg_per <= max_avg_threshold
            and XList_ru + XList_utility >= current_min_util
            and round(XList_prob, 3) >= user_prob_threshold
        ):
            new_lists: list[PNUList] = list()
            for j in range(i + 1, len(lists)):
                YList: PNUList = lists[j]
                x = XList.items.difference(PList.items)
                y = YList.items.difference(PList.items)
                key = frozenset(x | y)
                twu_value = eucs_dict.get(key, -1)
                if twu_value >= current_min_util:
                    ZList = utility_list_construct(
                        PList, XList, YList, current_min_util, user_prob_threshold
                    )
                    if ZList and round(XList_prob, 3) >= user_prob_threshold:
                        new_lists.append(ZList)
            if new_lists:
                phui_searching_procedure(
                    XList,
                    new_lists,
                    current_min_util,
                    min_per_threshold,
                    max_per_threshold,
                    min_avg_threshold,
                    max_avg_threshold,
                    user_prob_threshold,
                    database,
                    eucs_dict,
                    item_transId_dict,
                    k,
                    topk_queue,
                )

In [20]:
def calculate_dynamic_upper_bound_v2(
    Y: PNUList,
    X: PNUList,
    utility_array: list[int],
    item_index_dict: dict[Item, int],
) -> int:
    x = None
    for item in X.items:
        if item not in Y.items:
            x = item
            break

    if x is None:
        return Y.get_ru() + Y.get_pu()  
    return (
        Y.get_ru() + Y.get_pu() + get_value_from_bin(x, utility_array, item_index_dict)
    )

In [ ]:
def phui_searching_procedure_plus(
    PList: PNUList,
    lists: list[PNUList],
    current_min_util: int,
    min_per_threshold: int,
    max_per_threshold: int,
    min_avg_threshold: int,
    max_avg_threshold: int,
    user_prob_threshold: float,
    db: list[Transaction],
    utility_array: list[int],
    item_index_dict: dict[Item, int],
    eucs_dict: dict[frozenset[Item], int],
    item_transId_dict: dict[Item, list[int]],
    k: int,
    topk_queue: PriorityQueue,
) -> None:
    for i in range(0, len(lists)):
        XList: PNUList = lists[i]
        x_utility = XList.get_utility()
        x_prob = XList.get_pro()
        remaining_utility = XList.get_ru()
        min_per, max_per, avg_per = find_min_max_avg_periodic_of_item_set(
            XList.items, db, item_transId_dict
        )
        if (
            min_per >= min_per_threshold
            and max_per <= max_per_threshold
            and avg_per >= min_avg_threshold
            and avg_per <= max_avg_threshold
            and x_utility >= current_min_util
            and round(x_prob, 3) >= user_prob_threshold
        ):
            topk_queue.push(x_utility, XList.items)
            if len(topk_queue.heap) == k:
                current_min_util = topk_queue.sort()[k - 1][0]
        if (
            max_per <= max_per_threshold
            and avg_per <= max_avg_threshold
            and remaining_utility + x_utility >= current_min_util
            and round(x_prob, 3) >= user_prob_threshold
        ):
            new_lists: list[PNUList] = list()
            for j in range(i + 1, len(lists)):
                YList: PNUList = lists[j]
                x = XList.items.difference(PList.items)
                y = YList.items.difference(PList.items)
                twu_value = eucs_dict.get(frozenset(x | y), -1)
                if twu_value >= current_min_util:
                    du = calculate_dynamic_upper_bound_v2(
                        YList, XList, utility_array, item_index_dict
                    )
                    if du >= current_min_util:
                        if isinstance(YList, MList):
                            ZList: PNUList = utility_list_construct(
                                YList.prefix,
                                XList,
                                YList,
                                current_min_util,
                                user_prob_threshold,
                            )
                            if ZList:
                                new_lists.append(ZList)
                        else:
                            ZList: PNUList = utility_list_construct(
                                PList,
                                XList,
                                YList,
                                current_min_util,
                                user_prob_threshold,
                            )
                            if ZList:
                                new_lists.append(ZList)
                    else:
                        if isinstance(YList, MList):
                            ZMlist : MList= generate_mlist(XList, YList, None, du)
                            new_lists.append(ZMlist)
                        else:
                            ZMlist: MList = generate_mlist(XList, YList, PList, du)
                            new_lists.append(ZMlist)
            phui_searching_procedure_plus(
                XList,
                new_lists,
                current_min_util,
                min_per_threshold,
                max_per_threshold,
                min_avg_threshold,
                max_avg_threshold,
                user_prob_threshold,
                db,
                utility_array,
                item_index_dict,
                eucs_dict,
                item_transId_dict,
                k,
                topk_queue,
            )

In [22]:
def create_pnu_lists(trans_dict: dict[Item, set[Transaction]]):
    pnu_list = []
    for item, transactions in trans_dict.items():
        utility_values_list = []
        pnu = PNUList({item}, utility_values_list)
        for trans in transactions:
            pro = trans.get_probability_of_item_set({item})
            pu = trans.get_positive_utility_of_item_set({item})
            nu = trans.get_negative_utility_of_item_set({item})
            ru = trans.get_remaining_utility_of_item_set([item])
            utility_values = Utilities(trans.id, pro, pu, nu, ru)
            utility_values_list.append(utility_values)
        pnu_list.append(pnu)
    return pnu_list

In [ ]:
def topk_mining_based_on_PHUI(
    database: list[Transaction],
    item_list: list[Item],
    k: int,
    min_per_threshold: int,
    max_per_threshold: int,
    min_avg_threshold: int,
    max_avg_threshold: int,
    min_prob: float,
    is_plus: bool,
) -> PriorityQueue:
    t1 = datetime.datetime.now()
    # Create priority to contains top-k HUI
    topk_queue = PriorityQueue(k)
    user_prob_threshold = min_prob * len(database)

    # Create a utility array & and a dictionary to search
    prob_arr, twu_arr, utility_arr, item_index_dict = create_prob_twu_utility_bin_array(
        item_list, database
    )
    positive_utility_arr = [u for u in utility_arr if u > 0]
    # First update min_util = the k-th highest utility value (using RIU strategy)
    current_min_util: int = priu_pruning(positive_utility_arr, k)

    # Create a list that contains all items is unqualified
    removed_list: set[Item] = set()

    # Create a diction contains item: list[transaction_id]
    item_transId_dict: dict[Item, list[int]] = create_item_transId_dict(
        item_list, database
    )

    for i in range(len(item_list)):
        _, max_per, avg_per = find_min_max_avg_periodic_of_item_set(
            {item_list[i]}, database, item_transId_dict
        )
        if (
            max_per <= max_per_threshold
            and avg_per <= max_avg_threshold
            and twu_arr[i] >= current_min_util
            and round(prob_arr[i], 3) >= user_prob_threshold
        ):
            item_list[i].twu = twu_arr[i]
        else:
            removed_list.add(item_list[i])
    # Remove unqualified item
    new_distinct_items = [item for item in item_list if item not in removed_list]

    # Sort item list by order
    new_distinct_items = sort_items_by_twu_and_utility(new_distinct_items)

    # Remove unqualified items from transaction
    for trans in database:
        for item in removed_list:
            trans.trans_items_dict.pop(item, None)
            try:
                trans.trans_items_list.remove(item)
            except ValueError:
                pass
        trans.sort_trans_items_by_twu_and_utility()

    # Convert database - list[Transaction] into dict[Item, set[Transaction]]
    trans_dict: dict[Item, set[Transaction]] = create_transaction_dictionary(
        new_distinct_items, database
    )
    # Create list[AbstractList],
    pnu_lists: list[PNUList] = create_pnu_lists(trans_dict)

    # Create EUCST dict that contain twu > current_min_util of all 2-item-set
    eucst_dict: dict[frozenset[Item], int] = create_eucst_dict(
        new_distinct_items, database, trans_dict, current_min_util
    )

    # Create CUDM dict that contain utility of all 2-item-set
    cudm_dict: dict[frozenset[Item], int] = create_liu_dict(
        new_distinct_items, trans_dict
    )

    # Update and increase current_min_util
    if cudm_dict:
        cud: int = pliue_strategy(cudm_dict, k, current_min_util)
        # Update current min_util = the k-th highest utility in CUDM dict
        current_min_util = max(current_min_util, cud)

    covl: list[int] = covl_construct(
        new_distinct_items, eucst_dict, utility_arr, item_index_dict, trans_dict
    )
    # Update and increase current_min_util
    if covl:
        current_min_util = max(current_min_util, covl[min(len(covl), k) - 1])

    eucst_dict: dict[frozenset, int] = update_eucst_dict(eucst_dict, current_min_util)
    root = PNUList({}, list())
    t2 = datetime.datetime.now()
    print("phase 1 util: " + str(current_min_util))
    print("phase 1 take time: " + str(t2 - t1))
    if is_plus:
        phui_searching_procedure_plus(
            root,
            pnu_lists,
            current_min_util,
            min_per_threshold,
            max_per_threshold,
            min_avg_threshold,
            max_avg_threshold,
            user_prob_threshold,
            database,
            utility_arr,
            item_index_dict,
            eucst_dict,
            item_transId_dict,
            k,
            topk_queue,
        )
    else:
        phui_searching_procedure(
            root,
            pnu_lists,
            current_min_util,
            min_per_threshold,
            max_per_threshold,
            min_avg_threshold,
            max_avg_threshold,
            user_prob_threshold,
            database,
            eucst_dict,
            item_transId_dict,
            k,
            topk_queue,
        )
    # t3 = datetime.datetime.now()
    # print("phase 2 util: " + str(current_min_util))
    # print("phase 2 take time: " + str(t3 - t2))
    # print(t2 - t1)
    return topk_queue

<!-- **Tree Base** -->


In [24]:
# class PHUNode:
#     def __init__(
#         self,
#         item_set: set[Item] = None,
#         utility: int = 0,
#         ru: int = 0,
#         prob: float = 0.0,
#         children: list["PHUNode"] = None,
#         parent: "PHUNode" = None,
#     ):
#         self.item_set = item_set if item_set is not None else set()
#         self.utility = utility
#         self.ru = ru
#         self.prob = prob
#         self.children = children if children is not None else []
#         self.parent = parent

#     def add_child(self, child: "PHUNode") -> None:
#         self.children.append(child)

#     def is_leaf(self) -> bool:
#         return len(self.children) == 0

#     def get_total_utility(self) -> int:
#         total = self.utility
#         for child in self.children:
#             total += child.get_total_utility()
#         return total

#     def get_right_sibling(self):
#         if self.parent is None:
#             return []

#         siblings = self.parent.children

#         if not siblings or self not in siblings:
#             return []

#         current_index = siblings.index(self)
#         if current_index < len(siblings) - 1:
#             return siblings[current_index + 1 :]
#         else:
#             return []

#     def is_htwui(
#         self, database: list[Transaction], min_util: int, user_prob_threshold: float
#     ) -> bool:
#         p, twu = calculate_probability_and_twu_of_item_set(self.item_set, database)
#         return twu >= min_util and round(p, 3) >= user_prob_threshold

#     def __repr__(self) -> str:
#         return f"{self.item_set}"

#     def print_children(self, level=0):
#         print("  " * level + f"{self.item_set}")
#         for child in self.children:
#             child.print_children(level + 1)

In [ ]:
# def build_subtree(
#     node_x: PHUNode,
#     database: list[Transaction],
#     current_min_util: int,
#     user_prob_threshold: float,
#     k: int,
#     topk_queue: PriorityQueue,
# ):
#     if not node_x.is_htwui(database, current_min_util, user_prob_threshold):
#         return
#     right_siblings = node_x.get_right_sibling()
#     generates: list[PHUNode] = list()
#     for node_y in right_siblings:
#         if node_y.is_htwui(database, current_min_util, user_prob_threshold):
#             xy = (node_x.item_set - node_y.item_set).union(
#                 node_y.item_set - node_x.item_set
#             )
#            # xy_twu = calculate_transaction_weight_utility(xy, database)
#             xy_twu = 0
#             # EUCP Pruning
#             if xy_twu >= current_min_util:
#                 z_item_set = node_x.item_set.union(node_y.item_set)
#                 z_p, z_u, z_ru, twu = calculate_utilities_of_item_set(
#                     z_item_set, database
#                 )
#                 node_z = PHUNode(z_item_set, z_u, z_ru, z_p, [], node_x)
#                 node_x.add_child(node_z)
#                 if round(z_p, 3) >= user_prob_threshold:
#                     generates.append(node_z)
#                     if z_u >= current_min_util:
#                         if frozenset(z_item_set) not in {
#                             frozenset(item[1]) for item in topk_queue.heap
#                         }:
#                             topk_queue.push(z_u, z_item_set)
#                             if len(topk_queue.heap) == k:
#                                 current_min_util = topk_queue.sort()[k - 1][0]
#     for gen_node in generates:
#         build_subtree(
#             gen_node, database, current_min_util, user_prob_threshold, k, topk_queue
#         )

In [26]:
# def build_tree(
#     database: list[Transaction], item_list: list[Item], k: int, min_prob: float
# ):
#     min_util = 0
#     user_prob_threshold = min_prob * len(database)
#     topk_queue: PriorityQueue = PriorityQueue(k)
#     positive_list: list[Item] = [item for item in item_list if item.utility >= 0]
#     current_min_util: int = priu_pruning(positive_list, min_util, database)
#     removed_list = set()
#     for item in item_list:
#         prob, twu = calculate_probability_and_twu_of_item_set({item}, database)
#         if round(prob, 3) >= user_prob_threshold and item.twu >= current_min_util:
#             item.twu = twu
#         else:
#             removed_list.add(item)
#     # Remove unqualified item
#     new_distinct_items = [item for item in item_list if item not in removed_list]
#     # sort item list by order
#     new_distinct_items = sort_items_by_twu_and_utility(new_distinct_items)
#     root: PHUNode = PHUNode()
#     for item in new_distinct_items:
#         new_node = PHUNode({item}, 0, 0, 0, list(), root)
#         root.add_child(new_node)
#         utility = calculate_utility_of_item_set_in_database({item}, database)
#         if utility >= current_min_util:
#             if frozenset({item}) not in {
#                 frozenset(item[1]) for item in topk_queue.heap
#             }:
#                 topk_queue.push(utility, {item})
#                 if len(topk_queue.heap) == k:
#                     current_min_util = topk_queue.sort()[k - 1][0]

#     liu_dict: dict[frozenset[Item], int] = create_liu_dict(new_distinct_items, database)
#     pliue_util = pliue_strategy(liu_dict, k, min_util)
#     current_min_util = max(current_min_util, pliue_util)
#     for child in root.children:
#         build_subtree(
#             child, database, current_min_util, user_prob_threshold, k, topk_queue
#         )
#     return topk_queue

<!-- **TESTING** -->

<!-- **TESTCASE 1 - TOP-K WITHOUT PROBABILITY** -->

In [27]:
# a = Item("a", 4)
# b = Item("b", 3)
# c = Item("c", 1)
# d = Item("d", -1)
# e = Item("e", 2)

# t1_trans_items = [
#     TransItem(a, 1, 0.0),
#     TransItem(b, 2, 0.0),
#     TransItem(c, 2, 0.0),
#     TransItem(d, 1, 0.0),
# ]

# t2_trans_items = [
#     TransItem(a, 1, 0.0),
#     TransItem(b, 3, 0.0),
#     TransItem(c, 3, 0.0),
#     TransItem(e, 2, 0.0),
#     TransItem(d, 2, 0.0),
# ]

# t3_trans_items = [
#     TransItem(a, 1, 0.0),
#     TransItem(c, 6, 0.0),
#     TransItem(e, 3, 0.0),
# ]

# t4_trans_items = [TransItem(b, 3, 0.0), TransItem(e, 2, 0.0), TransItem(d, 5, 0.0)]

# t5_trans_items = [
#     TransItem(b, 1, 0.0),
#     TransItem(c, 5, 0.0),
#     TransItem(e, 4, 0.0),
#     TransItem(d, 1, 0.0),
# ]

# t6_trans_items = [
#     TransItem(c, 2, 0.0),
#     TransItem(e, 1, 0.0),
#     TransItem(d, 1, 0.0),
# ]

# t1 = Transaction(1, t1_trans_items)
# t2 = Transaction(2, t2_trans_items)
# t3 = Transaction(3, t3_trans_items)
# t4 = Transaction(4, t4_trans_items)
# t5 = Transaction(5, t5_trans_items)
# t6 = Transaction(5, t5_trans_items)

# item_list = [a, b, c, d, e]
# db1 = [t1, t2, t3, t4, t5]
# db2 = [t1, t2, t3, t4, t5]
# db3 = [t1, t2, t3, t4, t5]

In [28]:
a = Item("a", 3)
b = Item("b", 6)
c = Item("c", -3)
d = Item("d", 12)
e = Item("e", -5)
f = Item("f", -2)
g = Item("g", -1)

t1_trans_items = [
    TransItem(a, 5, 0.0),
    TransItem(b, 2, 0.0),
    TransItem(c, 1, 0.0),
    TransItem(d, 2, 0.0),
]

t2_trans_items = [
    TransItem(a, 1, 0.0),
    TransItem(c, 1, 0.0),
    TransItem(d, 1, 0.0),
    TransItem(g, 3, 0.0),
]

t3_trans_items = [
    TransItem(a, 1, 0.0),
    TransItem(c, 1, 0.0),
    TransItem(f, 1, 0.0),
]

t4_trans_items = [
    TransItem(a, 1, 0.0), 
    TransItem(f, 4, 0.0), 
    TransItem(g, 2, 0.0)]

t5_trans_items = [
    TransItem(a, 1, 0.0),
    TransItem(g, 2, 0.0),
]

t6_trans_items = [
    TransItem(b, 3, 0.0),
    TransItem(c, 2, 0.0),
    TransItem(d, 3, 0.0),
    TransItem(e, 1, 0.0),
]

t7_trans_items = [
    TransItem(c, 6, 0.0),
    TransItem(e, 4, 0.0),
]
t8_trans_items = [
    TransItem(e, 1, 0.0),
    TransItem(f, 3, 0.0),
]
t1 = Transaction(1, t1_trans_items)
t2 = Transaction(2, t2_trans_items)
t3 = Transaction(3, t3_trans_items)
t4 = Transaction(4, t4_trans_items)
t5 = Transaction(5, t5_trans_items)
t6 = Transaction(6, t6_trans_items)
t7 = Transaction(7, t7_trans_items)
t8 = Transaction(8, t8_trans_items)

item_list = [a, b, c, d, e, f, g]
db1 = [t1, t2, t3, t4, t5, t6, t7, t8]
db2 = [t1, t2, t3, t4, t5, t6, t7, t8]
db3 = [t1, t2, t3, t4, t5, t6, t7, t8]

In [29]:
k = 5
prob_threshold = 0.0
min_per_threshold = 1
max_per_threshold = 5
min_avg_threshold = 1
max_avg_threshold = 3

In [30]:
# t1 = datetime.datetime.now()
# rs1 = topk_mining_based_on_PHUI(
#     db1,
#     item_list,
#     k,
#     min_per_threshold,
#     max_per_threshold,
#     min_avg_threshold,
#     max_avg_threshold,
#     prob_threshold,
#     False,
# )
# rs1.print_items()
# t2 = datetime.datetime.now()
# print("topk_mining_based_on_PHUI(false): " + str(len(rs1.heap)))
# print("topk_mining_based_on_PHUI(false): " + str(t2 - t1))

In [31]:
# t1 = datetime.datetime.now()
# rs2 = topk_mining_based_on_PHUI(db2, item_list, k, min_per_threshold,
#     max_per_threshold,
#     min_avg_threshold,
#     max_avg_threshold, prob_threshold, True)
# rs2.print_items()
# t2 = datetime.datetime.now()
# print("topk_mining_based_on_PHUI(true): " + str(len(rs2.heap)))
# print("topk_mining_based_on_PHUI(true): " + str(t2 - t1))

In [32]:
print(db3)
t1 = datetime.datetime.now()
rs3 = topk_mining_based_on_EFIM(
    db3,
    item_list,
    k,
    min_per_threshold,
    max_per_threshold,
    min_avg_threshold,
    max_avg_threshold,
    prob_threshold,
)
rs3.print_items()
t2 = datetime.datetime.now()
print("topk_mining_based_on_EFIM: " + str(len(rs3.heap)))
print("topk_mining_based_on_EFIM: " + str(t2 - t1))

[Transaction(id=1, items=[a, b, c, d], quantities=[(5, 0.0), (2, 0.0), (1, 0.0), (2, 0.0)]), Transaction(id=2, items=[a, c, d, g], quantities=[(1, 0.0), (1, 0.0), (1, 0.0), (3, 0.0)]), Transaction(id=3, items=[a, c, f], quantities=[(1, 0.0), (1, 0.0), (1, 0.0)]), Transaction(id=4, items=[a, f, g], quantities=[(1, 0.0), (4, 0.0), (2, 0.0)]), Transaction(id=5, items=[a, g], quantities=[(1, 0.0), (2, 0.0)]), Transaction(id=6, items=[b, c, d, e], quantities=[(3, 0.0), (2, 0.0), (3, 0.0), (1, 0.0)]), Transaction(id=7, items=[c, e], quantities=[(6, 0.0), (4, 0.0)]), Transaction(id=8, items=[e, f], quantities=[(1, 0.0), (3, 0.0)])]
phase 1 util: 27
phase 1 take time: 0:00:00
phase 2 util: 27
phase 2 take time: 0:00:00
{b, d}: 90
{b, c, d}: 81
{d}: 72
{c, d}: 60
{b}: 30
topk_mining_based_on_EFIM: 5
topk_mining_based_on_EFIM: 0:00:00


<!-- **TESTCASE 2 - TOP-K WITH PROBABILITY** -->

In [33]:
a = Item("a", 6)
b = Item("b", 7)
c = Item("c", 1)
d = Item("d", -5)
e = Item("e", 3)

t1_trans_items = {TransItem(b, 3, 0.85), TransItem(c, 1, 1.0), TransItem(d, 2, 0.70)}

t2_trans_items = {
    TransItem(a, 1, 1.0),
    TransItem(b, 1, 0.60),
    TransItem(c, 3, 0.75),
    TransItem(e, 1, 0.40),
}

t3_trans_items = {
    TransItem(a, 1, 0.55),
    TransItem(b, 2, 0.60),
    TransItem(c, 4, 1.0),
    TransItem(d, 1, 0.90),
    TransItem(e, 5, 0.40),
}

t4_trans_items = {TransItem(b, 3, 0.90), TransItem(d, 1, 0.45)}

t5_trans_items = {
    TransItem(a, 4, 1.0),
    TransItem(c, 3, 0.85),
    TransItem(d, 2, 0.70),
    TransItem(e, 2, 0.45),
}

t1 = Transaction(1, t1_trans_items)
t2 = Transaction(2, t2_trans_items)
t3 = Transaction(3, t3_trans_items)
t4 = Transaction(4, t4_trans_items)
t5 = Transaction(5, t5_trans_items)

item_list = [a, b, c, d, e]
db1 = [t1, t2, t3, t4, t5]
db2 = [t1, t2, t3, t4, t5]
db3 = [t1, t2, t3, t4, t5]

In [34]:
# k = 10
# prob_threshold = 0.01
# min_per_threshold = 1
# max_per_threshold = 75
# min_avg_threshold = 1
# max_avg_threshold = 75

In [35]:
# t1 = datetime.datetime.now()
# rs1 = topk_mining_based_on_PHUI(db1, item_list, k, prob_threshold, False)
# rs1.print_items()
# t2 = datetime.datetime.now()
# print("topk_mining_based_on_PHUI(false): " + str(len(rs1.heap)))
# print("topk_mining_based_on_PHUI(false): " + str(t2 - t1))

In [36]:
# t1 = datetime.datetime.now()
# rs2 = topk_mining_based_on_PHUI(db2, item_list, k, prob_threshold, True)
# rs2.print_items()
# t2 = datetime.datetime.now()
# print("topk_mining_based_on_PHUI(true): " + str(len(rs2.heap)))
# print("topk_mining_based_on_PHUI(true): " + str(t2 - t1))

In [37]:
# t1 = datetime.datetime.now()
# rs3 = topk_mining_based_on_EFIM(db3, item_list, k, prob_threshold)
# rs3.print_items()
# t2 = datetime.datetime.now()
# print("topk_mining_based_on_EFIM: " + str(len(rs3.heap)))
# print("topk_mining_based_on_EFIM: " + str(t2 - t1))

<!-- **TESTCASE 3 - RANDOM** -->

In [38]:
item_list1 = [Item(f"i{i+1}", random.randint(-10, 10)) for i in range(129)]
item_list2 = list(item_list1)
item_list3 = list(item_list1)
db1 = list()
db2 = list()
db3 = list()
for transaction_id in range(1, 10001):
    trans_items = list()
    selected_items = random.sample(item_list1, random.randint(43, 43))
    for item in selected_items:
        quantity = random.randint(1, 10)
        probability = round(random.uniform(0.30, 0.99), 2)
        trans_item = TransItem(item, quantity, probability)
        trans_items.append(trans_item)
    transaction = Transaction(transaction_id, trans_items)
    db1.append(transaction)
    db2.append(transaction)
    db3.append(transaction)

In [ ]:
k = 10
prob_threshold = 0.0001
min_per_threshold = 1
max_per_threshold = 500
min_avg_threshold = 1
max_avg_threshold = 500

In [40]:
t1 = datetime.datetime.now()
rs1 = topk_mining_based_on_PHUI(
    db1,
    item_list1,
    k,
    min_per_threshold,
    max_per_threshold,
    min_avg_threshold,
    max_avg_threshold,
    prob_threshold,
    False,
)
rs1.print_items()
t2 = datetime.datetime.now()
print("topk_mining_based_on_PHUI(false): " + str(len(rs1.heap)))
print("topk_mining_based_on_PHUI(false): " + str(t2 - t1))

phase 1 util: 150520
phase 1 take time: 0:00:16.013909
{i52}: 188400
{i28}: 187910
{i49}: 182320
{i9}: 178830
{i93}: 178360
{i89}: 164286
{i111}: 162279
{i123}: 151168
{i66}: 150520
topk_mining_based_on_PHUI(false): 9
topk_mining_based_on_PHUI(false): 0:15:16.446143


In [41]:
t1 = datetime.datetime.now()
rs2 = topk_mining_based_on_PHUI(
    db2,
    item_list2,
    k,
    min_per_threshold,
    max_per_threshold,
    min_avg_threshold,
    max_avg_threshold,
    prob_threshold,
    True,
)
rs2.print_items()
t2 = datetime.datetime.now()
print("topk_mining_based_on_PHUI(true): " + str(len(rs2.heap)))
print("topk_mining_based_on_PHUI(true): " + str(t2 - t1))

phase 1 util: 150520
phase 1 take time: 0:00:54.164914
{i52}: 188400
{i28}: 187910
{i49}: 182320
{i9}: 178830
{i93}: 178360
{i89}: 164286
{i111}: 162279
{i123}: 151168
{i66}: 150520
topk_mining_based_on_PHUI(true): 9
topk_mining_based_on_PHUI(true): 0:23:21.578819


In [42]:
t1 = datetime.datetime.now()
rs3 = topk_mining_based_on_EFIM(
    db3,
    item_list3,
    k,
    min_per_threshold,
    max_per_threshold,
    min_avg_threshold,
    max_avg_threshold,
    prob_threshold,
)
rs3.print_items()
t2 = datetime.datetime.now()
print("topk_mining_based_on_EFIM: " + str(len(rs3.heap)))
print("topk_mining_based_on_EFIM: " + str(t2 - t1))

phase 1 util: 150520
phase 1 take time: 0:00:14.580771
phase 2 util: 150520
phase 2 take time: 1:14:31.957192
{i52}: 188400
{i28}: 187910
{i49}: 182320
{i9}: 178830
{i93}: 178360
{i89}: 164286
{i111}: 162279
{i123}: 151168
{i66}: 150520
topk_mining_based_on_EFIM: 9
topk_mining_based_on_EFIM: 1:14:46.562905


<!-- **DEBUGGING** -->